#### 고급 최적화
* 1. 제안-평가-등록 패러다임: <br/>
`maximize`는 내부적으로 `suggest`, `probe` 및 `register` 메서드를 래핑해 한번에 처리되기 때문에 `최적화 루프에 더 많은 제어가 필요한 경우` 사용할 수 있음.

#### 라이브러리 불러오기

In [6]:
from bayes_opt import UtilityFunction
from bayes_opt import BayesianOptimization

In [4]:
SEED = 1

#### 제안-평가-등록 패러다임

##### 기본적인 흐름

* 1. 대상함수 지정

In [1]:
# 대상함수 정의
def black_box_function(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

* 2. 대상함수를 특정하지 않는 채, 옵티마이저 지정

In [5]:
# 대상함수의 평가가 최적화객체에 의해 수행되지 않음.
# 이 함수의 평가가 다른 기계에서도 수행되거나, 다른 언어로 작성되었을 수 있기 때문
# `probe` 또는 `maximize` 메서드를 호출하지 않는 한 최적화객체는 대상함수를 무시함

optimizer = BayesianOptimization(
    f=None,     # 실행할 대상 함수를 직접 지정하지 않음
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=SEED
)

* 3. 최적화에 필요한 탐색 및 활용전략 설정

In [8]:
# 최적화에 적합한 다양한 탐색 및 활용전략 설정
utility = UtilityFunction(
    kind='ucb', # 탐색과 활용 사이의 균형 유지, 불확실한 지점을 더 많이 탐색
    kappa=2.5,  # UCB전략에서 얼마의 불확실성을 고려할지 지정
                # 값이 높으면 탐색을, 값이 낮으면 활용을 강조함
    xi=0.0      # 탐색전략에서 사용되는 파라미터로 얼마나 큰 개선을 고려할지 조정
                # 값이 높으면 큰 개선을 값이 낮으면 작은 개선을 탐색함
)

* 4. 다음에 탐색할 매개변수 조합을 확인

In [10]:
# 옵티마이저는 언제는 suggest인자로 utility를 이용해,
# 옵티마이저가 다음으로 탐색할 매개변수 조합을 제안할 수 있음
next_point_to_probe = optimizer.suggest(utility)
print(f'Next point to probe is: {next_point_to_probe}')

Next point to probe is: {'x': -1.9995425007306205, 'y': -1.1860045642089614}


* 5. 해당 시점에서 대상 함수 평가: 이 부분은 반복 실행됨

In [12]:
# 이렇게 제안된 지점에서 함수를 자유롭게 평가할 수 있음
target = black_box_function(**next_point_to_probe)
print(f'Found the target value to be: {target}')

Found the target value to be: -7.776786166970474


* 6. 해당 시점의 관찰된 대상값을 옵티마이저에 등록

In [13]:
# 마지막으로 할 일은, 관찰된 대상값을 옵티마이저에 알리는 것(등록)임.
optimizer.register(
    params=next_point_to_probe,
    target=target
)

##### 최대화 루프
위 4~6 단계들을 반복함으로써 최대화 메서드의 내부를 구현

In [14]:
for _ in range(5):
    next_point = optimizer.suggest(utility)
    target = black_box_function(**next_point)
    optimizer.register(params=next_point, target=target)
    print(target, next_point)
    
print(optimizer.max)

-6.855238338708555 {'x': 1.971861291575097, 'y': 2.9917332616332803}
-6.843971497794797 {'x': 1.9645795986017864, 'y': 2.996095813970973}
-4.860694575404144 {'x': 1.364072789628231, 'y': 3.0}
-3.806908887203212 {'x': 0.8982810736084849, 'y': 3.0}
-0.4627160776045711 {'x': 0.4304199308674709, 'y': 2.1302454426878317}
{'target': -0.4627160776045711, 'params': {'x': 0.4304199308674709, 'y': 2.1302454426878317}}


#### 불연속 매개변수 처리

In [17]:
# 대상함수 생성
def func_with_discrete_params(x, y, d):
    # 'd'값의 불연속 값(정수) 여부 체크
    assert type(d) == int
    
    return ((x + y + d) // (1 + d)) / (1 + (x + y) ** 2)

def function_to_be_optimized(x, y, w):
    d = int(w)
    return func_with_discrete_params(x, y, d)